<a href="https://colab.research.google.com/github/RohannnCodes/STSCI5050-Final-Project/blob/main/logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving merged_final.csv to merged_final.csv


In [2]:
import pandas as pd
import numpy as np

In [39]:
df = pd.read_csv('merged_final.csv', index_col=0)
df["Winner"] = (df["Team1"] == df["WINNER_TEAM"]).astype(int)
df["Home"] = (df["Team1"] == df["HOME_TEAM"]).astype(int)
df.head(50)

,Team1,Team2,AVG_ORAPM_Team1,AVG_DRAPM_Team1,AVG_ORAPM_Team2,AVG_DRAPM_Team2,AVG_RANK_Team1,AVG_RANK_Team2,season,GAME_ID,WINNER_SIDE,WINNER_TEAM,WINNER_TEAM_ID,HOME_TEAM,HOME_TEAM_ID,HOME_TEAM_WON,Winner,Home
GameID,,,,,,,,,,,,,,,,,,
21600005,IND,DAL,0.613900,-0.249140,0.411300,0.197460,237.600000,192.000000,2016-17,21600005,HOME,IND,1610612754,IND,1610612754,1,1,1
21600021,PHX,OKC,0.291440,-0.154980,0.306320,0.874840,265.200000,182.000000,2016-17,21600021,HOME,OKC,1610612760,OKC,1610612760,1,0,0
21600033,POR,DEN,0.992732,-0.280065,0.883160,-0.656640,227.678788,343.600000,2016-17,21600033,AWAY,POR,1610612757,DEN,1610612743,0,1,0
21600039,WAS,MEM,1.183892,-0.205005,0.510772,0.662675,211.678788,159.478788,2016-17,21600039,HOME,MEM,1610612763,MEM,1610612763,1,0,0
21600049,MIA,SAC,-0.493140,0.098440,0.064280,-0.237820,344.000000,314.400000,2016-17,21600049,HOME,MIA,1610612748,MIA,1610612748,1,1,1
21600060,NOP,MEM,0.050860,0.569620,0.925772,0.424315,249.400000,140.878788,2016-17,21600060,HOME,MEM,1610612763,MEM,1610612763,1,0,0
21600063,POR,PHX,0.904392,-0.459725,0.041860,-0.516960,262.478788,352.600000,2016-17,21600063,HOME,PHX,1610612756,PHX,1610612756,1,0,0
21600075,NOP,PHX,0.017760,0.583320,0.113820,-0.407600,265.800000,325.000000,2016-17,21600075,AWAY,PHX,1610612756,NOP,1610612740,0,0,1
21600090,MIL,DAL,0.564240,-0.467720,-0.178060,0.942520,273.400000,167.400000,2016-17,21600090,HOME,DAL,1610612742,DAL,1610612742,1,0,0


In [40]:
df_team1 = df.copy(deep=True)
df_team1["ORAPM"] = df_team1["AVG_ORAPM_Team1"] - df_team1["AVG_ORAPM_Team2"]
df_team1["DRAPM"] = df_team1["AVG_DRAPM_Team1"] - df_team1["AVG_DRAPM_Team2"]
df_team1["RELATIVE_RANK"] = (df_team1["AVG_RANK_Team1"] > df_team1["AVG_RANK_Team2"]).astype(int)
df_team1 = df_team1.drop(df_team1.columns[2:16], axis=1)
df_team1.head()

,Team1,Team2,Winner,Home,ORAPM,DRAPM,RELATIVE_RANK
GameID,,,,,,,
21600005,IND,DAL,1,1,0.202600,-0.446600,1
21600021,PHX,OKC,0,0,-0.014880,-1.029820,1
21600033,POR,DEN,1,0,0.109572,0.376575,0
21600039,WAS,MEM,0,0,0.673120,-0.867680,1
21600049,MIA,SAC,1,1,-0.557420,0.336260,1


In [41]:
df_team2 = df.copy(deep=True)
df_team2["ORAPM"] = df_team2["AVG_ORAPM_Team2"] - df_team2["AVG_ORAPM_Team1"]
df_team2["DRAPM"] =  df_team2["AVG_DRAPM_Team2"] - df_team2["AVG_DRAPM_Team1"]
df_team2["RELATIVE_RANK"] = (df_team2["AVG_RANK_Team1"] <= df_team2["AVG_RANK_Team2"]).astype(int)
df_team2["Team1"], df_team2["Team2"] = df_team2["Team2"], df_team2["Team1"]
df_team2["Winner"] = 1 - df_team2["Winner"]
df_team2["Home"] = 1 - df_team2["Home"]
df_team2 = df_team2.drop(df_team2.columns[2:16], axis=1)

df_team2.head()


,Team1,Team2,Winner,Home,ORAPM,DRAPM,RELATIVE_RANK
GameID,,,,,,,
21600005,DAL,IND,0,0,-0.202600,0.446600,0
21600021,OKC,PHX,1,1,0.014880,1.029820,0
21600033,DEN,POR,0,1,-0.109572,-0.376575,1
21600039,MEM,WAS,1,1,-0.673120,0.867680,0
21600049,SAC,MIA,0,0,0.557420,-0.336260,0


In [42]:
LR_df = pd.concat([df_team1, df_team2], axis=0)
LR_df.head(10)

,Team1,Team2,Winner,Home,ORAPM,DRAPM,RELATIVE_RANK
GameID,,,,,,,
21600005,IND,DAL,1,1,0.202600,-0.446600,1
21600021,PHX,OKC,0,0,-0.014880,-1.029820,1
21600033,POR,DEN,1,0,0.109572,0.376575,0
21600039,WAS,MEM,0,0,0.673120,-0.867680,1
21600049,MIA,SAC,1,1,-0.557420,0.336260,1
21600060,NOP,MEM,0,0,-0.874912,0.145305,1
21600063,POR,PHX,0,0,0.862532,0.057235,0
21600075,NOP,PHX,0,1,-0.096060,0.990920,0
21600090,MIL,DAL,0,0,0.742300,-1.410240,1


In [43]:
features = LR_df[["Home", "ORAPM", "DRAPM", "RELATIVE_RANK"]]
target = LR_df["Winner"]

In [44]:
import numpy as np

X_np = features.to_numpy().astype("float32")
y_np = target.to_numpy().astype("float32").reshape(-1, 1)

In [45]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled_np = scaler.fit_transform(X_np).astype("float32")

In [47]:
from sklearn.model_selection import train_test_split

X_train_np, X_test_np, y_train_np, y_test_np = train_test_split(
    X_scaled_np, y_np, test_size=0.2, random_state=42, shuffle=True
)

In [48]:
import torch

X_train = torch.from_numpy(X_train_np).float()
X_test = torch.from_numpy(X_test_np).float()

y_train = torch.from_numpy(y_train_np).float()
y_test = torch.from_numpy(y_test_np).float()

Logistic Regression Model

In [49]:
import torch
torch.__version__, torch.cuda.is_available()

('2.9.0+cu126', True)

In [50]:
import torch
print(torch.cuda.get_device_name(0))

Tesla T4


In [51]:
import torch
import torch.nn as nn
import torch.optim as optim

In [52]:
class LogisticRegression:
    def __init__(self, dim):
        self.w = torch.randn(dim, 1, requires_grad=True)
        self.b = torch.zeros(1, requires_grad=True)

    def sigmoid(self, z):
        return 1 / (1 + torch.exp(-z))

    def predict_proba(self, X):
        z = X @ self.w + self.b
        return self.sigmoid(z)

    def loss(self, X, y):
        p = self.predict_proba(X)
        loss = - (y * torch.log(p + 1e-12) + (1 - y) * torch.log(1 - p + 1e-12)).mean()
        return loss

In [55]:
model = LogisticRegression(dim=X_train.shape[1])
optimizer = torch.optim.SGD([model.w, model.b], lr=0.1)

for epoch in range(500):
    optimizer.zero_grad()
    loss = model.loss(X_train, y_train)
    loss.backward()
    optimizer.step()

    if epoch % 50 == 0:
        print(f"epoch {epoch}, loss={loss.item():.4f}")

epoch 0, loss=0.7376
epoch 50, loss=0.6894
epoch 100, loss=0.6795
epoch 150, loss=0.6756
epoch 200, loss=0.6734
epoch 250, loss=0.6722
epoch 300, loss=0.6716
epoch 350, loss=0.6712
epoch 400, loss=0.6710
epoch 450, loss=0.6708


In [57]:
with torch.no_grad():
    probs = model.predict_proba(X_test)
    preds = (probs > 0.5).float()

accuracy = (preds == y_test).float().mean().item()

In [58]:
print(accuracy)

0.675000011920929


In [59]:
with torch.no_grad():
    train_probs = model.predict_proba(X_train)
    train_preds = (train_probs > 0.5).float()

train_accuracy = (train_preds == y_train).float().mean().item()
print("Training Accuracy:", train_accuracy)

Training Accuracy: 0.5822784900665283
